# Process
In this notebook, we'll practice how to combine datasets.  We'll use the Fitbit dataset from Kaggle.com found here: https://www.kaggle.com/datasets/arashnic/fitbit?resource=download

In [ ]:
# Import libraries:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Workshop Functions
import sys
sys.path.append('..')
from Wksp722_functions import * 

In [ ]:
Activity = pd.read_csv('dailyActivity_merged.csv')
Sleep = pd.read_csv('sleepDay_merged.csv')

In [ ]:
Activity.head()

In [ ]:
Sleep.head()

Both tables have an 'Id' column which corresponds to the user, and an ActivityDate/SleepDay that corresponds to the date of the recording.  Neither are unique.  But in combination, they form unique qualifiers.  

First, let's split SleepDay into 2 columns, one for date, and another for time.  

#### Sleep Table Processing

In [ ]:
Sleep[['ActivityDate', 'Time']] = Sleep['SleepDay'].str.split(' ',expand=True)
Sleep.drop(['SleepDay','Time'], axis=1, inplace=True) # Drop unecessary columns

In [ ]:
Sleep.info()

In [ ]:
# Convert 'Id' column to String
Sleep['Id'] = Sleep['Id'].apply(str)

# Combine Id and ActivityDate columns
Sleep['Key'] = Sleep['ActivityDate'].str.cat(Sleep['Id'], sep=' ')

In [ ]:
Sleep.head()

#### Activity Table Processing

In [ ]:
Activity.head()

In [ ]:
Activity['Id'] = Activity['Id'].apply(str) #convert Id column to String

Activity['Key'] = Activity['ActivityDate'].str.cat(Activity['Id'], sep=' ')
Activity.head()

In [ ]:
CombinedTable = Activity.merge(Sleep, how='inner', on='Key')
CombinedTable.head()

Now we can filter and group across all columns, thereby expanding the range of inquiries we can explore.  Let's start with a correlation heatmap:

In [ ]:
# first, let's drop some tables that won't likely be correlated
temp1 = CombinedTable.drop(['Id_x','ActivityDate_x','Id_y','ActivityDate_y'],axis=1)

sns.heatmap(temp1.corr(numeric_only=True))

When looking between the tables, we see that there is a very strong correlation between **TotalMinutesAsleep** and **SedentaryMinutes**

In [ ]:
sns.lmplot(data=CombinedTable, x='TotalMinutesAsleep',y='SedentaryMinutes')
plt.show()

We can take a deeper dive by binning the **SedentaryMinutes** and then plotting the average for both columns for each of these bins.  This provides a discrete view of the above scatter plot

In [ ]:
temp2 = CombinedTable[['TotalMinutesAsleep','SedentaryMinutes']].copy()
temp2.head()

In [ ]:
temp2['SedentaryMinutesGroups'] = pd.qcut(temp2['SedentaryMinutes'],10)
temp2.head()

In [ ]:
temp2.groupby('SedentaryMinutesGroups').mean()
summaryTable = temp2.groupby('SedentaryMinutesGroups').mean()
display(summaryTable)

In [ ]:
sns.lmplot(data=summaryTable, x='TotalMinutesAsleep',y='SedentaryMinutes')
plt.show()